In [1]:
#!pip3 install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html

In [2]:
#!pip3 install -q "monai-weekly[nibabel, skimage, pillow, tensorboard, gdown, ignite, torchvision, itk, tqdm, lmdb, psutil,  openslide, pandas, einops, transformers, mlflow, matplotlib, tensorboardX, tifffile, imagecodecs]"

In [3]:
#!pip3 install h5py

In [4]:
from monai.config import print_config
print_config()

MONAI version: 0.9.dev2211
Numpy version: 1.21.5
Pytorch version: 1.10.2+cu113
MONAI flags: HAS_EXT = False, USE_COMPILED = False
MONAI rev id: 9c0d89cb7c774cba53242b9a3981fd02e75f3902
MONAI __file__: C:\Program Files\Python37\lib\site-packages\monai\__init__.py

Optional dependencies:
Pytorch Ignite version: 0.4.8
Nibabel version: 3.2.2
scikit-image version: 0.19.2
Pillow version: 9.0.1
Tensorboard version: 2.8.0
gdown version: 4.4.0
TorchVision version: 0.11.3+cu113
tqdm version: 4.63.0
lmdb version: 1.3.0
psutil version: 5.9.0
pandas version: 1.3.5
einops version: 0.4.1
transformers version: 4.17.0
mlflow version: 1.24.0

For details about installing the optional dependencies, please visit:
    https://docs.monai.io/en/latest/installation.html#installing-the-recommended-dependencies



In [5]:
import numpy as np

from monai.utils import first, set_determinism
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    ScaleIntensityRanged,
    Spacingd,

)
from monai.networks.nets import UNet
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, DataLoader, Dataset, decollate_batch
from monai.config import print_config
from monai.apps import download_and_extract
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import os
import glob



from monai.transforms import (
    LoadImage, LoadImaged, EnsureChannelFirstd,
    Resized,  Compose
)
from monai.config import print_config
import re
from juliacall import Main as jl

[juliapkg] Locating Julia ^1.6
[juliapkg] Using Julia 1.7.2 at C:\Julia-1.7.2\bin\julia.EXE
[juliapkg] Using Julia project at C:\Users\jakub\.julia\environments\pyjuliapkg
[juliapkg] Installing packages:
           julia> import Pkg
           julia> Pkg.add([Pkg.PackageSpec(name="PythonCall", uuid="6099a3de-0909-46bc-b1f4-468b9a2dfc0d"), Pkg.PackageSpec(name="MedPipe3D", uuid="80492ae4-ba08-4770-a16e-2cec06b8739e")])
           julia> Pkg.resolve()


In [12]:
import juliapkg
juliapkg.resolve()
import MedPipe3D

ModuleNotFoundError: No module named 'MedPipe3D'

In [6]:
from monai.transforms import *

set_determinism(seed=0)
val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"]),
        EnsureChannelFirstd(keys=["image", "label"]),
        Spacingd(keys=["image", "label"], pixdim=(
            1.0, 1.0, 1.0), mode=("bilinear", "nearest")),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        EnsureTyped(keys=["image", "label"]),
    ]
)


In [ ]:
data_dir = "D:\\dataSets\\CTORGmini\\"

train_images = sorted(
    glob.glob(os.path.join(data_dir, "volumes 0-49", "*.nii.gz")))

train_labels = sorted(
    glob.glob(os.path.join(data_dir, "labels", "*.nii.gz")))

data_dicts = [
    {"image": image_name, "label": label_name}
    for image_name, label_name in zip(train_images, train_labels)
]
data_dicts

[{'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-0.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-0.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-1.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-1.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-2.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-2.nii.gz'},
 {'image': 'D:\\dataSets\\CTORGmini\\volumes 0-49\\volume-3.nii.gz',
  'label': 'D:\\dataSets\\CTORGmini\\labels\\labels-3.nii.gz'}]

In [ ]:
image0Transformed=val_transforms(data_dicts[0])
image0Transformed

{'image': tensor([[[[-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           ...,
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.]],
 
          [[-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           ...,
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.]],
 
          [[-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-3024., -3024., -3024.,  ..., -3024., -3024., -3024.],
           [-30

In [36]:
def getNumberFromFilePath(dat):
    string = str(dat['image_meta_dict']['filename_or_obj'])
    numb = re.search('.nii', string).span()[0] ## this does what you wanted
    describingNumber = string[(numb-2):(numb)].replace("-","")
    return int(describingNumber)

In [37]:
import h5py
f = h5py.File("D:\\dataSets\\forMainHDF5\\forHausdorffTestsBig.hdf5", "w")

In [38]:
#we are looking for a mask with given organ and save a boolean that will make later benchmarks easier
def processGivenMaskAndSaveSum(grp, name, dat,numb ):
    labelBoolTensor =torch.where( dat['label']==numb, 1, 0).bool()
    summ= torch.sum(labelBoolTensor)
    #if summ>0:
    labelBoolDataset = grp.create_dataset(name, data=  torch.flatten(labelBoolTensor).detach().numpy())
    labelBoolDataset.attrs['sum']=  summ
    #return summ

In [39]:
from monai.metrics import HausdorffDistanceMetric
from torch.utils import benchmark

def hdToTest(a, b):
    hd = HausdorffDistanceMetric(percentile=1.0)
    hd(y_pred=a.to(torch.device("cuda")), y=b.to(torch.device("cuda")))  # callable to add metric to the buffer
    metric = hd.aggregate().item()
    return metric




#now we are exetunig GPU Hausdorff distance measure between two arbitrary organs
#arrSums - sums summarizing other label mask entries  maskNumb - current entry
#maskNumb - number of a mask from 0 to 7 from ct org 
def getSomeHausdorffDist(grp, name, dat,numb , arrSums  ):
    for maskNumb in range(1,7) :
        if (maskNumb!=numb and arrSums[numb]>0) :
            labelBoolTensorThis =torch.where(dat['label']==numb, 1, 0).bool()
            labelBoolTensorOther =torch.where(dat['label']==maskNumb, 1, 0).bool()
            t0 = benchmark.Timer(
                stmt='hdToTest(a, b)',
                setup='from __main__ import hdToTest',
                globals={'a':labelBoolTensorThis , 'b':labelBoolTensorOther })
            time = t0.timeit(100)
            metrValue = hdToTest(labelBoolTensorThis,labelBoolTensorOther)
            labelBoolDataset = grp[str(numb)]
            labelBoolDataset.attrs['comparedWith']= maskNumb
            labelBoolDataset.attrs['HausdorfGPUmetrValue']=metrValue
            labelBoolDataset.attrs['HausdorfGPUmetrtime']=str(time)
            sizz=labelBoolTensorThis.size()
            labelBoolDataset.attrs['xDim']=sizz[0]
            labelBoolDataset.attrs['yDim']=sizz[1]
            labelBoolDataset.attrs['zDim']=sizz[2]

            break



In [40]:
check_ds = Dataset(data=data_dicts, transform=val_transforms)
check_loader = DataLoader(check_ds, batch_size=1)

for dat in check_loader:
    describingNumber = getNumberFromFilePath(dat)
    #group collecting
    grp = f.create_group(str(describingNumber))
    
    arrSums= np.zeros(7)
    for maskNumb in range(1,7) :
        locSum= processGivenMaskAndSaveSum(grp, str(maskNumb), dat,maskNumb )
        arrSums[maskNumb]=maskNumb
    for maskNumb in range(1,7) :
        getSomeHausdorffDist(grp, str(describingNumber), dat,maskNumb , arrSums  )
    print(describingNumber)
    pass

KeyboardInterrupt: 

In [ ]:
f.close()